In [1]:
import api_nyt as nyt
import api_wikipedia as wiki
import manage_articles as mng
import statistics
import datetime
import pickle
import timeseries
import TextPreprocessing as txt
from datetime import date
import graphics
import matplotlib.pyplot as plt
import matching
import time
import word

articles = mng.load_articles("nyt2019.json")

In [2]:
#headlines = mng.getWordCounts(articles,'headline')
#with open("wordCount2019.txt", "wb") as fp:
#    pickle.dump(headlines, fp)
#top = mng.getTopWordsForWeek(headlines)
#distinctWords = mng.getDistinctWords(headlines)

headlines = []
with open("wordCount2019.txt", "rb") as fp:
    headlines = pickle.load(fp)
distinctWords = mng.getDistinctWords(headlines)

countsPerWeek = []
for w in distinctWords:
    countsPerWeek.append((w,mng.getCountPerWeek(headlines,w)))

#ts = []
#for c in countsPerWeek:
#    ts.append(timeseries.Timeseries(c[0],c[1]))

words = []
for c in countsPerWeek:
    words.append(word.Word(c[0],ts_articles=timeseries.Timeseries(c[1])))

In [3]:
words = sorted(words, key=lambda x: sum(x.ts_articles.getCounts()), reverse=True)

In [4]:
top_amount = 50

keywords = []
for i in range(top_amount):
    keywords.append(words[i].keyword)

m = matching.groupmatch(keywords, articles)
for key in m.keys():
    #Following line from 
    #https://stackoverflow.com/questions/7125467/find-object-in-list-that-has-attribute-equal-to-some-value-that-meets-any-condi
    word = next((x for x in words if x.keyword == key), None)
    word.coocKeywords = m[key]['query']
    word.wikipediaSite = m[key]['link'][1]

words_with_link = [w for w in words if w.wikipediaSite != ""]
for word in words_with_link:
    word.ts_wiki = timeseries.parseWikipediaCounts(wiki.get_counts(words[0].wikipediaSite, words[0].ts_articles.getStartDate(), words[0].ts_articles.getEndDate()))

In [15]:
statistics.getCorrelation(words[8].ts_articles.getCounts(),words[8].ts_wiki.getCounts())

(0.29034577432510716, 0.03679506470936405)